In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent=dataset["Purchased"]

In [9]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [10]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.0, 0.1, 0.5, 1.0, 10],
    'fit_prior': [True, False],
    'class_prior': [None], 
    'force_alpha': [True, False]
    }

grid = GridSearchCV(MultinomialNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the model
grid.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.0, 0.1, 0.5, 1.0, 10],
                         'class_prior': [None], 'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 0.0, 'class_prior': None, 'fit_prior': True, 'force_alpha': True}: 0.49240100865535336


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[85  0]
 [49  0]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.63      1.00      0.78        85
           1       0.00      0.00      0.00        49

    accuracy                           0.63       134
   macro avg       0.32      0.50      0.39       134
weighted avg       0.40      0.63      0.49       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.5834333733493396

In [18]:
table=pd.DataFrame.from_dict(re)

In [19]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010399,0.002762,0.032389,0.006193,0.0,None,True,True,"{'alpha': 0.0, 'class_prior': None, 'fit_prior...",0.509779,0.567088,0.590313,0.635834,0.557004,0.572004,0.041305,1
1,0.013153,0.002234,0.024493,0.012318,0.0,None,True,False,"{'alpha': 0.0, 'class_prior': None, 'fit_prior...",0.509779,0.567088,0.590313,0.635834,0.557004,0.572004,0.041305,1
2,0.007566,0.000533,0.009428,0.001057,0.0,None,False,True,"{'alpha': 0.0, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,11
3,0.010537,0.002285,0.012235,0.002777,0.0,None,False,False,"{'alpha': 0.0, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,11
4,0.009089,0.001587,0.011998,0.001570,0.1,None,True,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509779,0.567088,0.590313,0.635834,0.557004,0.572004,0.041305,1
5,0.008115,0.001000,0.010498,0.001055,0.1,None,True,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509779,0.567088,0.590313,0.635834,0.557004,0.572004,0.041305,1
6,0.010873,0.003408,0.012686,0.001870,0.1,None,False,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,11
7,0.009813,0.001548,0.009944,0.000539,0.1,None,False,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,11
8,0.010138,0.001230,0.012788,0.005882,0.5,None,True,True,"{'alpha': 0.5, 'class_prior': None, 'fit_prior...",0.509779,0.567088,0.590313,0.635834,0.557004,0.572004,0.041305,1
9,0.008841,0.001260,0.009558,0.001693,0.5,None,True,False,"{'alpha': 0.5, 'class_prior': None, 'fit_prior...",0.509779,0.567088,0.590313,0.635834,0.557004,0.572004,0.041305,1


In [20]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age:34
Salary:20000
Sex Male 0 or 1:1


In [21]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]]) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[0]


C:\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
